In [1]:
import helix
from helix.client import Query
from helix.types import Payload
from typing import List

db = helix.Client(local=True, port=6969)

Exception: [31m[HELIX][0m No helix server found at 'http://0.0.0.0:6969'

## Check Configs

In [37]:
import json
import os
from pathlib import Path

CONFIG_PATH = "helixdb-cfg"
CONFIG_FILE = "config.hx.json"
QUERIES_FILE = "queries.hx"
SCHEMA_FILE = "schema.hx"
DEFAULT_CONFIG = {
    "vector_config": {
        "m": 16,
        "ef_construction": 128,
        "ef_search": 768
    },
    "graph_config": {
        "secondary_indices": []
    },
    "db_max_size_gb": 10
}

helix_dir = Path(os.path.dirname(os.path.curdir)).resolve()
os.makedirs(os.path.join(helix_dir, CONFIG_PATH), exist_ok=True)

if not Path(os.path.join(helix_dir, CONFIG_PATH, CONFIG_FILE)).exists():
    configs = DEFAULT_CONFIG
    path = os.path.join(helix_dir, CONFIG_PATH, CONFIG_FILE)
    with open(path, "w") as f:
        json.dump(configs, f, indent=4)
    print("Config file created")
else:
    path = os.path.join(helix_dir, CONFIG_PATH, CONFIG_FILE)
    with open(path, "r") as f:
        configs = json.load(f)
    print("Config file loaded")

Config file created


## Create Schema

In [ ]:
def create_node(node_type:str, properties:dict = {}) -> str:
    output = ""
    output += "N::" + node_type + " {\n"
    for key, value in properties.items():
        output += "    " + str(key) + ": " + str(value) + "\n"
    output += "}\n"
    return output

print(create_node("User", {"name": "String", "age": "U32"}))

N::User {
    name: String
    age: U32
}



In [61]:
def create_edge(edge_type:str, from_node:str, to_node:str, properties:dict = {}) -> str:
    output = ""
    output += "E::" + edge_type + " {\n"
    output += "    From: " + from_node + "\n"
    output += "    To: " + to_node + "\n"
    output += "    Properties: {\n"
    for key, value in properties.items():
        output += "        " + str(key) + ": " + str(value) + "\n"
    output += "    }\n"
    output += "}\n"
    return output

print(create_edge("Follows", "User", "User", {"since": "I32"}))

E::Follows {
    From: User
    To: User
    Properties: {
        since: I32
    }
}



In [62]:
def create_vector(vector_type:str, properties:dict = {}) -> str:
    output = ""
    output += "V::" + vector_type + " {\n"
    for key, value in properties.items():
        output += "    " + str(key) + ": " + str(value) + "\n"
    output += "}\n"
    return output

print(create_vector("User", {"name": "String", "age": "U32"}))

V::User {
    name: String
    age: U32
}



## Helix CLI

### Deploy

In [2]:
import subprocess
import re
import pexpect

# Run command in background
deploy_db = subprocess.Popen(
    ['helix', 'deploy', '--path', 'helixdb-cfg', '--port', '6969'],
    stdout=subprocess.PIPE,
    text=True
)

output = []

# Read output line by line
for line in deploy_db.stdout:
    process_line = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    line = process_line.sub('', line)
    output.append(line.strip())

# Wait for process to complete
deploy_db.wait()

instance_id = [out for out in output if out.startswith("Instance ID:")][0].split("Instance ID: ")[1].split(" (running)")[0]
print('\n'.join(output))
print(f"Instance ID: {instance_id}")

Successfully compiled 2 query files
Successfully built Helix
⢹ Starting Helix instance

Port 6969 is in use, using port 6970 instead
Successfully transpiled queries
Successfully wrote queries file
Instance ID: dd795160-0899-4c6b-bd41-cc6657178f10 (running)
└── Label:
└── Port: 6970
└── Available endpoints:
└── /create_user
└── /get_users
└── /create_post
└── /get_followed_users_posts
└── /create_follow
└── /get_posts
└── /get_posts_by_user
└── /get_followed_users
Instance ID: dd795160-0899-4c6b-bd41-cc6657178f10


Successfully started Helix instance


### Stop

In [ ]:
stop_db = pexpect.spawn(
    ' '.join(['helix', 'stop', instance_id]),
)

stop_out = stop_db.before.decode('utf-8').split('\n')

print(stop_out)

for line in stop_out:
    process_line = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    line = process_line.sub('', line)
    print(line)

AttributeError: 'NoneType' object has no attribute 'decode'

### Delete

In [122]:
delete_db = pexpect.spawn(
    ' '.join(['helix', 'delete', instance_id]),
)

delete_db.expect("Are you sure you want to delete")
delete_db.sendline("y")
delete_db.expect(pexpect.EOF)

del_out = delete_db.before.decode('utf-8').split('\n')

for line in del_out:
    process_line = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    line = process_line.sub('', line)
    print(line)

 the instance and its data? (y/n): y
Deleted Helix instance
Deleted Helix instance data



## Create RAG Query